In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

np.set_printoptions(threshold=100)

In [2]:
# Read data from file
train_df = pd.read_csv('data/mnist-in-csv/mnist_train.csv', sep=',')
test_df = pd.read_csv('data/mnist-in-csv/mnist_test.csv', sep=',')

train_data = train_df.iloc[:,1:].to_numpy(dtype='float32')
train_target = train_df.iloc[:,0].to_numpy(dtype='int8')
train_target = OneHotEncoder(sparse=False).fit_transform(train_target.reshape(-1, 1))

test_data = test_df.iloc[:,1:].to_numpy(dtype='float32')
test_target = test_df.iloc[:,0].to_numpy(dtype='int8')
test_target = OneHotEncoder(sparse=False).fit_transform(test_target.reshape(-1, 1))

/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'"

In [3]:
# 重要パラメータの定義
# データの性質関連
TRAIN_SIZE = 20000
TEST_SIZE = 5000
STEPS_PER_DATA = 28
NUM_INPUT_NODES = 28
NUM_OUTPUT_NODES = 10

# リザバー関連
LEAK_RATE=0.3
SPECTRAL_RADIUS = 1.2
NUM_RESERVOIR_NODES = 1200
# BIAS = 1.0

In [4]:
train_data = train_data[:TRAIN_SIZE].reshape((-1,28))
test_data = test_data[:TEST_SIZE].reshape((-1,28))
train_target = train_target[:TRAIN_SIZE]
test_target = test_target[:TEST_SIZE]

In [5]:
# 手書き文字画像を20x20にトリミング

NUM_INPUT_NODES = 20
STEPS_PER_DATA = 20

train_indices = np.zeros(20*TRAIN_SIZE, dtype='int')
for i in range(TRAIN_SIZE):
    train_indices[i*20 : (i+1)*20] = np.arange((i*28+4),(i*28+24))
    
train_data = train_data[train_indices,4:24]

test_indices = np.zeros(20*TEST_SIZE, dtype='int')
for i in range(TEST_SIZE):
    test_indices[i*20 : (i+1)*20] = np.arange((i*28+4),(i*28+24))
    
test_data = test_data[test_indices,4:24]

In [6]:
train_data.shape

(400000, 20)

In [7]:
test_data.shape

(100000, 20)

In [8]:
train_target.shape

(20000, 10)

In [9]:
test_target.shape

(5000, 10)

In [10]:
train_target = [i for i in train_target for _ in range(STEPS_PER_DATA)]
#test_target = [i for i in test_target for _ in range(STEPS_PER_DATA)]

In [11]:
# example of activator
def ReLU(x):
    return np.maximum(0, x)

# 行の最大値を1、それ以外を0にする関数
def max_to_one(row):
    index = np.argmax(row)
    row = np.zeros(len(row))
    row[index] = 1
    return row

# ***この関数は不要になった***
# parentは2次元numpy配列で、childが足したい行（1次元numpy配列）
# np.append(parent, [child], axis=0)に等しい
# def numpy_append(parent, child):
#     parent_list = parent.tolist()
#     child_list = child.tolist()
#     parent_list.append(child_list)
#     parent = np.asarray(parent_list)
#     return parent

class ReservoirNetWork: # 入れるtrain_data, test_data, targetは全てnumpy配列

    def __init__(self, train_data, num_input_nodes, num_reservoir_nodes, num_output_nodes, leak_rate=0.1, activator=np.tanh):
        self.train_data = train_data
        self.log_reservoir_nodes = np.zeros((len(train_data), num_reservoir_nodes), dtype='float32') # reservoir層のノードの状態を記録

        # init weights
        self.weights_input = self._generate_input_weights(num_input_nodes, num_reservoir_nodes).astype('float32')
        self.weights_reservoir = self._generate_reservoir_weights(num_reservoir_nodes).astype('float32')
        self.weights_output = np.zeros((num_reservoir_nodes, num_output_nodes), dtype='float32')

        # それぞれの層のノードの数
        self.num_input_nodes = num_input_nodes
        self.num_reservoir_nodes = num_reservoir_nodes
        self.num_output_nodes = num_output_nodes

        self.leak_rate = leak_rate # 漏れ率
        self.activator = activator # 活性化関数

    # reservoir層のノードの次の状態を取得
    def _get_next_reservoir_nodes(self, input, current_state):
        next_state = (1 - self.leak_rate) * current_state
        next_state += self.leak_rate * (input @ self.weights_input + current_state @ self.weights_reservoir)
        return self.activator(next_state)

    # 出力層の重みを更新
    def _update_weights_output(self, target, lambda0):
        # Ridge Regression
        E_lambda0 = np.identity(self.num_reservoir_nodes) * lambda0
        inv_x = np.linalg.inv(self.log_reservoir_nodes.T @ self.log_reservoir_nodes + E_lambda0)
        # update weights of output layer
        self.weights_output = ((inv_x @ self.log_reservoir_nodes.T) @ target)

    # 学習する
    def train(self, lambda0=0.1):
        for i in range(len(self.train_data)):
            tr = self.train_data[i]
            current_state = self.log_reservoir_nodes[max(0,i-1)]
                
            self.log_reservoir_nodes[i] = self._get_next_reservoir_nodes(tr, current_state)
            # self.log_reservoir_nodes = numpy_append(self.log_reservoir_nodes, self._get_next_reservoir_nodes(tr, current_state))
            # self.log_reservoir_nodes = np.append(self.log_reservoir_nodes, [self._get_next_reservoir_nodes(tr, current_state)], axis=0)
            
            if i % 1000 == 0:
                print('training data no. {}\n'.format(i))
                
        # まとめて行列計算で重みを更新
        self._update_weights_output(train_target, lambda0)
        
    # 予測する
    def predict(self, test_data):
        reservoir_nodes = self.log_reservoir_nodes[-1] # 訓練の結果得た最後の内部状態を取得
        
        predict_output = np.zeros((len(test_data), (NUM_OUTPUT_NODES)), dtype='float32')
        reduced_predict_output = np.zeros(( int(len(test_data)/STEPS_PER_DATA), NUM_OUTPUT_NODES), dtype='float32')
        tmp_array = np.zeros(NUM_OUTPUT_NODES)
        tmp_count = 0
        
        for i in range(len(test_data)):
            te = test_data[i]
            reservoir_nodes = self._get_next_reservoir_nodes(te, reservoir_nodes) # 内部状態更新
            predict_output[i] = max_to_one( self.get_output(reservoir_nodes) ) # 内部状態を読み出して出力を得る、出力が最大のところを1とする
            # predict_output = numpy_append(predict_output, output)
            # predict_output = np.append(predict_output, [output], axis=0)
            # predict_output = predict_output[1:]
            tmp_array += predict_output[i]
            
            if(i%STEPS_PER_DATA == STEPS_PER_DATA - 1):
                reduced_predict_output[tmp_count] = tmp_array
                # reduced_predict_output = numpy_append(reduced_predict_output, tmp_array)
                # reduced_predict_output = np.append(reduced_predict_output, [tmp_array], axis=0)
                tmp_array = np.zeros(NUM_OUTPUT_NODES)
                tmp_count += 1    
                
        final_predict_output = [max_to_one(row) for row in reduced_predict_output]
        
        return final_predict_output, predict_output, reduced_predict_output

    # 内部状態から出力を計算
    def get_output(self, reservoir_nodes):
        return reservoir_nodes @ self.weights_output 

    #############################
    ##### private method ########
    #############################

    # 重みを0.1か0か-0.1で初期化したものを返す
    def _generate_input_weights(self, num_input_nodes, num_reservoir_nodes):
        return np.random.choice([-0.1, 0, 0.1], num_input_nodes*num_reservoir_nodes, p=[0.1, 0.8, 0.1]).reshape(num_input_nodes, num_reservoir_nodes)

    # Reservoir層の重みを初期化
    def _generate_reservoir_weights(self, num_nodes):
        weights = np.random.normal(0, 1, num_nodes * num_nodes)
        indices = np.random.choice(np.arange(len(weights)), replace=False, size=int(len(weights) * 0.9))
        weights[indices] = 0
        weights = weights.reshape([num_nodes, num_nodes])
        spectral_radius = max(abs(linalg.eigvals(weights)))
        return weights / spectral_radius * SPECTRAL_RADIUS


In [12]:
model = ReservoirNetWork(train_data=train_data,
    num_input_nodes=NUM_INPUT_NODES,
    num_reservoir_nodes=NUM_RESERVOIR_NODES,
    num_output_nodes=NUM_OUTPUT_NODES,
    leak_rate=LEAK_RATE)

model.train() # 訓練

final_predict_output, predict_output, reduced_predict_output = model.predict(test_data)


training data no. 0

training data no. 1000

training data no. 2000

training data no. 3000

training data no. 4000

training data no. 5000

training data no. 6000

training data no. 7000

training data no. 8000

training data no. 9000

training data no. 10000

training data no. 11000

training data no. 12000

training data no. 13000

training data no. 14000

training data no. 15000

training data no. 16000

training data no. 17000

training data no. 18000

training data no. 19000

training data no. 20000

training data no. 21000

training data no. 22000

training data no. 23000

training data no. 24000

training data no. 25000

training data no. 26000

training data no. 27000

training data no. 28000

training data no. 29000

training data no. 30000

training data no. 31000

training data no. 32000

training data no. 33000

training data no. 34000

training data no. 35000

training data no. 36000

training data no. 37000

training data no. 38000

training data no. 39000

training data

training data no. 320000

training data no. 321000

training data no. 322000

training data no. 323000

training data no. 324000

training data no. 325000

training data no. 326000

training data no. 327000

training data no. 328000

training data no. 329000

training data no. 330000

training data no. 331000

training data no. 332000

training data no. 333000

training data no. 334000

training data no. 335000

training data no. 336000

training data no. 337000

training data no. 338000

training data no. 339000

training data no. 340000

training data no. 341000

training data no. 342000

training data no. 343000

training data no. 344000

training data no. 345000

training data no. 346000

training data no. 347000

training data no. 348000

training data no. 349000

training data no. 350000

training data no. 351000

training data no. 352000

training data no. 353000

training data no. 354000

training data no. 355000

training data no. 356000

training data no. 357000

training dat

In [13]:
# 誤差
np.sum(np.absolute(final_predict_output - test_target)) * 0.5 / len(test_target)

0.328

In [14]:
test_target.tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [15]:
final_predict_output

[array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),


In [16]:
predict_output.tolist()

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,

In [17]:
reduced_predict_output.tolist()

[[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 18.0, 0.0, 0.0],
 [0.0, 6.0, 1.0, 3.0, 0.0, 1.0, 6.0, 1.0, 2.0, 0.0],
 [0.0, 12.0, 0.0, 0.0, 2.0, 0.0, 1.0, 5.0, 0.0, 0.0],
 [9.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 3.0],
 [4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 8.0],
 [0.0, 16.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0],
 [5.0, 2.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 1.0, 8.0],
 [1.0, 1.0, 0.0, 7.0, 0.0, 1.0, 4.0, 0.0, 3.0, 3.0],
 [3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 0.0, 6.0, 0.0],
 [5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 2.0, 0.0, 3.0],
 [16.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 0.0, 1.0, 2.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 3.0, 12.0],
 [12.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 2.0, 2.0],
 [0.0, 17.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 1.0, 2.0, 14.0, 0.0, 0.0, 1.0, 1.0],
 [3.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 1.0, 3.0, 9.0],
 [0.0, 5.0, 0.0, 7.0, 0.0, 0.0, 0.0, 7.0, 0.0, 1.0],
 [4.0, 0.0, 4.0, 3.0, 0.0, 4.0, 0.0,